In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

text= u"We started off asking what started Ale Adventures and what gave them the idea to blog about beer and breweries.\
Based out of Minnesota, Adam and Katie have taken Ale Adventures to 7 overseas breweries, 7 breweries in North Dakota, 24 breweries in Wisconsin, 7 breweries in Illinois, and 98 breweries in Minnesota. \
With a scoreboard of that size, we were curious what their take was on different beers throughout the world. \
Since Adam & Katie are based out of Minnesota, we wanted to know more about the brewery scene there and what some of their favorite places are.\
We also had some suggestions of places and breweries we have been as well, and shared some of our favorite places. \
We also wanted to get to know more about the creators and co-writers of Ale Adventures, and how they met, so we asked more about where and how they met and how that led into a mutual appreciation and passion for beer. \
Since we’re a travel blog, we were interested what got them traveling more out of the country to visit breweries. \
So we asked if beer was the origin of these trips, or if they simply selected countries and found some breweries when they were there. \
We were also interested in their perspective of international vs. national beers and if they noticed any major difference between the two. We thought Ale Adventures was a great beer blog and now we can confirm it also has some great creators writers! Definitely make sure to check out Ale Adventures blog or Instagram for great beer content! Welcome back to the Friends That Carry On Podcast! We're a group of friends that like to travel the world together. \
We want to share our passion for travel through travel tips and tricks we've picked up along away and stories of our adventures! For Episode 14,  Tony Price is our host and tchotchke holder, and he’s joined this week alongside Eric Fargo and Jim Scott. \
This episode, the 3 Friends will be discussing different things to do in Madrid, breaking down the planning, where to stay, and everything in between. \
Hold on tight for another great episode! Our tchotchke this episode is this tiled bull, made up of tiles from Madrid, brought by Eric. As a group of friends, we have all been together in Pamplona , flying in and out of Madrid, so after having flown in and out of there, we made it a goal for us to go and spend more time in Madrid. \
So first, we talked about traveling out there and tips for getting there, as well as the places we stayed and tips for that. \
We like to share our experiences of traveling and where we stayed, to give some insight on things that worked and didn’t work for us, so we can hopefully help to influence how you might travel or where you might stay. \
We then led into some of the tours we’ve taken in Madrid, first being a Walking Tour, with Sandman Tours, and the second being a food tour, or the Tapas Tour. \
We’ve talked about tours in the past, including the Walking Tour in Madrid specifically, but we always recommend tours when visiting a place. \
If you take a tour of the area the first or second day, it’s a great way to learn the area a little bit and get comfortable with your surroundings."

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))


Length of text: 3132 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

We started off asking what started Ale Adventures and what gave them the idea to blog about beer and breweries.Based out of Minnesota, Adam and Katie have taken Ale Adventures to 7 overseas breweries, 7 breweries in North Dakota, 24 breweries in Wisc


In [5]:
# The unique characters in the file
uniquevocab = sorted(set(text))
print ('{} unique characters'.format(len(uniquevocab)))

57 unique characters


In [6]:
# Vectorize the text
# Before training, 
# we need to map strings to a numerical representation. 
# Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Mapping from unique characters to indices
char2index = {u:i for i, u in enumerate(uniquevocab)}
index2char = np.array(uniquevocab)

text_as_int = np.array([char2index[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2index[char]))
print('  ...\n}')


{
  ' ' :   0,
  '!' :   1,
  '&' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  '1' :   7,
  '2' :   8,
  '3' :   9,
  '4' :  10,
  '7' :  11,
  '8' :  12,
  '9' :  13,
  'A' :  14,
  'B' :  15,
  'C' :  16,
  'D' :  17,
  'E' :  18,
  'F' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'We started of' ---- characters mapped to int ---- > [30 35  0 48 49 31 47 49 35 34  0 45 36]


In [10]:
# /* 
# The prediction task
# Given a character, or a sequence of characters, what is the most probable next character? This is the task 
# we're training the model to perform. The input to the model will be a sequence of characters, and we train 
# the model to predict the output—the following character at each time step.

# Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters 
# computed until this moment, what is the next character?

# Create training examples and targets
#--------------------------------------------
# 1. Next divide the text into example sequences. 
# 2. Each input sequence will contain seq_length characters from the text.

# 3. For each input sequence, the corresponding targets contain the same length of text, except shifted one character
#    to the right.

# 4. So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". 
# The input sequence would be "Hell", and the target sequence "ello".

# To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream 
# of character indices.
# */


In [67]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

print(examples_per_epoch)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])
  print(i.numpy())

31
W
30
e
35
 
0
s
48
t
49


In [12]:
# repr(object)
# Return a string containing a printable representation of an object. 
# For many types, this function makes an attempt to return a string that would yield an 
# object with the same value when passed to eval(), otherwise the representation is a string 
# enclosed in angle brackets that contains the name of the type of the object together with additional 
# information often including the name and address of the object. A class can control what this function 
# returns for its instances by defining a __repr__() method.

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))

'We started off asking what started Ale Adventures and what gave them the idea to blog about beer and '
'breweries.Based out of Minnesota, Adam and Katie have taken Ale Adventures to 7 overseas breweries, 7'
' breweries in North Dakota, 24 breweries in Wisconsin, 7 breweries in Illinois, and 98 breweries in M'
'innesota. With a scoreboard of that size, we were curious what their take was on different beers thro'
'ughout the world. Since Adam & Katie are based out of Minnesota, we wanted to know more about the bre'


In [74]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [75]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(index2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(index2char[target_example.numpy()])))

Input data:  'We started off asking what started Ale Adventures and what gave them the idea to blog about beer and'
Target data: 'e started off asking what started Ale Adventures and what gave them the idea to blog about beer and '


In [61]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(index2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(index2char[target_idx])))

Step    0
  input: 30 ('W')
  expected output: 35 ('e')
Step    1
  input: 35 ('e')
  expected output: 0 (' ')
Step    2
  input: 0 (' ')
  expected output: 48 ('s')
Step    3
  input: 48 ('s')
  expected output: 49 ('t')
Step    4
  input: 49 ('t')
  expected output: 31 ('a')


In [51]:
# Batch size
#BATCH_SIZE = 10

steps_per_epoch = examples_per_epoch//BATCH_SIZE

#print(examples_per_epoch)
#print(steps_per_epoch)

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
#BUFFER_SIZE = 10000


#dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

#print(dataset.take(1))

31
3
<DatasetV1Adapter shapes: ((10, 64, 100), (10, 64, 100)), types: (tf.int32, tf.int32)>


In [62]:
# Length of the vocabulary in chars
vocab_size = len(uniquevocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [63]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [64]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [65]:
model = build_model(
  vocab_size = len(uniquevocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [66]:
print(dataset.take(1))

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>


ValueError: Shape (256, 100) must have rank at least 3

In [28]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           14592     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_1 (Dense)              (64, None, 57)            58425     
Total params: 4,008,249
Trainable params: 4,008,249
Non-trainable params: 0
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

NameError: name 'example_batch_predictions' is not defined

In [ ]:
sampled_indices


In [ ]:
print("Input: \n", repr("".join(index2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(index2char[sampled_indices])))

In [34]:
# Train the model

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


NameError: name 'target_example_batch' is not defined

In [ ]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=3

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

In [ ]:
# Generate the text
tf.train.latest_checkpoint(checkpoint_dir)


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"Ale Adventures and what"))
